<a href="https://colab.research.google.com/github/FREYDER18/SYS/blob/main/DETECTOR_DE_GENERO%20taller%202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit librosa joblib pyngrok yt-dlp gdown

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.3/174.3 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 953.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 3.3 MB/s eta 0:00:00


In [2]:
%%writefile app.py
import streamlit as st
import librosa
import librosa.display
import numpy as np
import joblib
import os
import yt_dlp
import gdown  # Para descargar desde Google Drive
import matplotlib.pyplot as plt

#https://drive.google.com/file/d/12IPvTtEBg0joAP4MseWcR8oiQLr0raT6/view?usp=sharing

st.set_page_config(page_title="Detector de Género Musical", page_icon="🎵")
st.title("🎶 Detector de Género Musical: Reguetón vs Salsa")

# 🔗 ID del archivo en Google Drive
FILE_ID = "12IPvTtEBg0joAP4MseWcR8oiQLr0raT6"
modelo_path = "/content/reggaeton_vs_salsa.pkl"

# 📥 Descargar modelo si no existe localmente
if not os.path.exists(modelo_path):
    try:
        st.info("📥 Descargando modelo desde Google Drive...")
        gdown.download(f"https://drive.google.com/uc?id={FILE_ID}", modelo_path, quiet=False)
    except Exception as e:
        st.error(f"❌ Error descargando el modelo: {e}")
        st.stop()

# ✅ Cargar modelo
try:
    modelo_dict = joblib.load(modelo_path)
    modelo = modelo_dict['modelo'] if isinstance(modelo_dict, dict) else modelo_dict
    st.success("✅ Modelo cargado correctamente.")
except Exception as e:
    st.error(f"❌ Error cargando el modelo: {e}")
    st.stop()

# 🔊 Descargar audio desde YouTube como MP3
def download_ytvid_as_mp3(video_url, name):
    output = f"/content/{name}"
    options = {
        'format': 'bestaudio/best',
        'outtmpl': output,
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
    }
    with yt_dlp.YoutubeDL(options) as ydl:
        try:
            ydl.download([video_url])
            final_path = f"{output}.mp3"
            return final_path if os.path.exists(final_path) else None
        except Exception as e:
            st.error(f"❌ Error al descargar audio: {e}")
            return None

# 🎛️ Extraer características (FFT normalizada)
def extraer_fft(file_path):
    try:
        y, sr = librosa.load(file_path, sr=22050)
        fft = np.abs(np.fft.fft(y))
        fft = fft[:len(fft)//2]

        # ⚠️ Normalizar como en el entrenamiento
        if np.max(fft) != 0:
            fft = fft / np.max(fft)

        if len(fft) < 120001:
            fft = np.pad(fft, (0, 120001 - len(fft)), mode='constant')
        else:
            fft = fft[:120001]

        return fft.reshape(1, -1), y, sr  # ← también retornamos la señal y sr
    except Exception as e:
        st.error(f"❌ Error al procesar el audio: {e}")
        return None, None, None

# 🧠 Diccionario de clases
CLASES = {
    1.0: "Reguetón",
    2.0: "Salsa"
}

# 🔗 Entrada del usuario
url = st.text_input("🔗 Ingresa el enlace de la canción (YouTube):")

if url:
    with st.spinner("⬇️ Descargando audio..."):
        mp3_file = download_ytvid_as_mp3(url, "audio")
        if not mp3_file:
            st.error("❌ El archivo MP3 no fue encontrado después de la descarga.")
            st.stop()

    # 🔊 Mostrar reproductor de audio
    st.subheader("🔊 Reproductor de audio")
    audio_bytes = open(mp3_file, 'rb').read()
    st.audio(audio_bytes, format="audio/mp3")

    with st.spinner("🎛️ Extrayendo características..."):
        features, y, sr = extraer_fft(mp3_file)
        if features is None:
            st.stop()

    # 📈 Visualizar forma de onda
    st.subheader("📉 Forma de onda")
    fig_wave, ax_wave = plt.subplots(figsize=(10, 2))
    librosa.display.waveshow(y, sr=sr, ax=ax_wave, color='mediumseagreen')
    ax_wave.set_xlabel("Tiempo (s)")
    ax_wave.set_ylabel("Amplitud")
    st.pyplot(fig_wave)

    # 📊 Visualizar espectro FFT
    st.subheader("📊 Espectro de Frecuencia (FFT)")
    fft_vals = np.abs(np.fft.fft(y))[:len(y)//2]
    freqs = np.fft.fftfreq(len(y), 1/sr)[:len(y)//2]
    fig_fft, ax_fft = plt.subplots(figsize=(10, 3))
    ax_fft.plot(freqs, fft_vals, color='coral')
    ax_fft.set_xlim(0, 8000)
    ax_fft.set_xlabel("Frecuencia (Hz)")
    ax_fft.set_ylabel("Magnitud")
    st.pyplot(fig_fft)

    with st.spinner("🔍 Clasificando género musical..."):
        try:
            pred = modelo.predict(features)[0]
            genero = CLASES.get(pred, "Desconocido")
            st.success(f"🎵 Género detectado: **{genero}**")
        except Exception as e:
            st.error(f"❌ Error al clasificar: {e}")


Writing app.py


In [3]:
from pyngrok import ngrok, conf
import os
import threading

# ⚠️ Matamos sesiones activas
ngrok.kill()

# ✅ Token ya configurado (no es necesario repetir si ya está)
ngrok.set_auth_token("2y8ReOP8WVMANp23qnojZmq2yH4_5dZ24aw6BG7qwRpf6TLbx")

# 🚀 Ejecutar Streamlit en segundo plano
def run():
    os.system("streamlit run app.py")

thread = threading.Thread(target=run)
thread.start()

# 🌍 Conectar a nuevo túnel
public_url = ngrok.connect(8501)
print(f"✅ App publicada: {public_url}")



✅ App publicada: NgrokTunnel: "https://2b2a-34-66-146-151.ngrok-free.app" -> "http://localhost:8501"
